In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np

In [2]:
train_datagen = ImageDataGenerator(
        rotation_range=360,
        width_shift_range=0.0,
        height_shift_range=0.0,
        rescale=1./255,
        shear_range=0.0,
        zoom_range=0.0,
        fill_mode="nearest")

train_generator = train_datagen.flow_from_directory("data/train",
                                                target_size = (300, 300),
                                                batch_size = 16,
                                                class_mode = 'categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory("data/test",
                                                 target_size=(300, 300),
                                                 batch_size=16,
                                                 class_mode='categorical')

Found 331 images belonging to 4 classes.
Found 83 images belonging to 4 classes.


In [3]:
model = Sequential()

model.add(Conv2D(16, (3, 3), input_shape = (300, 300, 3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units = 4, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 300, 300, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 37, 37, 64)        1

In [5]:
model.fit(train_generator,
                    steps_per_epoch=331//16,
                    validation_data=test_generator,
                    validation_steps=83//16,
                    epochs = 5)

Epoch 1/5
20/20 [==============================] - 15s 740ms/step - loss: 1.3126 - accuracy: 0.4000 - val_loss: 1.1600 - val_accuracy: 0.4750
Epoch 2/5
20/20 [==============================] - 18s 924ms/step - loss: 1.0306 - accuracy: 0.5683 - val_loss: 0.8501 - val_accuracy: 0.5250
Epoch 3/5
20/20 [==============================] - 22s 1s/step - loss: 0.8976 - accuracy: 0.5492 - val_loss: 0.7697 - val_accuracy: 0.6500
Epoch 4/5
20/20 [==============================] - 25s 1s/step - loss: 0.9508 - accuracy: 0.5397 - val_loss: 0.8223 - val_accuracy: 0.5750
Epoch 5/5
20/20 [==============================] - 21s 1s/step - loss: 0.8484 - accuracy: 0.5524 - val_loss: 0.7810 - val_accuracy: 0.6000


In [6]:
scores = model.evaluate(test_generator, steps=5)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

5/5 [==============================] - 1s 196ms/step - loss: 0.7703 - accuracy: 0.5875
accuracy: 58.75%


In [9]:
model.save('model/my_model.h5')

In [ ]:
output = model.predict_generator(test_generator, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_generator.class_indices)
print(output)